In [12]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
import pandas as pd
import json
import glob

FOLDER_PATH = "/vepfs/wcf/G/zecheng/my_repo/Long-Context-Data-Engineering/eval/results/llama-2-7b-80k_testPPL_64000/"
MODEL_NAME = "LLaMA 2 7B continue-trained on 5B tokens 80K length Per-source length upsampled data"
PRETRAINED_LEN=81920

# Path to the directory containing JSON results
folder_path = FOLDER_PATH
if("/" in folder_path):
    model_name = folder_path.split("/")[-2]
else: model_name = MODEL_NAME
print("model_name = %s" % model_name)

# Using glob to find all json files in the directory
json_files = glob.glob(f"{folder_path}*.json")

# List to hold the data
data = []
data2 = []

# Iterating through each file and extract the 3 columns we need
for file in json_files:
    with open(file, 'r') as f:
        json_data = json.load(f)
        # Extracting the required fields
        document_depth = json_data.get("depth_percent", None)
        context_length = json_data.get("context_length", None)
        # score = json_data.get("score", None)
        model_response = json_data.get("model_response", None).lower()
        needle = json_data.get("needle", None).lower()
        str_ppls = json_data.get("ppls", None)
        shift_st, shift_end = json_data.get("shift_st", None), json_data.get("shift_end", None)
        ppls = json.loads(str_ppls) if str_ppls is not None else None
        expected_answer = "eat a sandwich and sit in Dolores Park on a sunny day.".lower().split()
        score = len(set(model_response.split()).intersection(set(expected_answer))) / len(expected_answer)
        # Appending to the list
        """
        量化PPLs的变化指标
        1. 开头的PPL变化 bt_ppl_cg
        2. 结尾的PPL变化 end_ppl_cg
        3. 前半句话的PPL bt_ins_avg_ppl
        4. 后半句话的PPL ed_ins_avg_ppl
        """
        bt_ppl_cg = abs(sum(ppls[shift_st-2: shift_st]) - sum(ppls[shift_st: shift_st+2])) / 2
        end_ppl_cg = abs(sum(ppls[shift_end-2: shift_end]) - sum(ppls[shift_end: shift_end+2])) / 2
        bt_ins_avg_ppl = sum(ppls[:shift_st]) / shift_st
        ed_ins_avg_ppl = sum(ppls[shift_end: ]) / shift_end
        data.append({"Document Depth": document_depth, "Context Length": context_length,
            "Score": score, "PPLs": ppls, "bt_ppl_cg": bt_ppl_cg, "end_ppl_cg": end_ppl_cg,  
            "bt_ins_avg_ppl": bt_ins_avg_ppl, "ed_ins_avg_ppl": ed_ins_avg_ppl,
        })


# Creating a DataFrame
df = pd.DataFrame(data)
df2 = pd.DataFrame(data2)
locations = list(df["Context Length"].unique())
locations.sort()
for li, l in enumerate(locations):
    if(l > PRETRAINED_LEN): break
pretrained_len = li

print(df.head())
print("Overall score %.3f" % df["Score"].mean())

sns.jointplot(x="bt_ppl_cg", y="end_ppl_cg", data=df)

plt.show()

model_name = llama-2-7b-80k_testPPL_64000
   Document Depth  Context Length     Score  \
0            44.0            7513  0.916667   
1            11.0           46590  0.916667   
2            78.0           49846  0.916667   
3            89.0           33564  0.916667   
4            67.0           53103  0.916667   

                                                PPLs    bt_ppl_cg  \
0  [78.047, 4.219, 1.009, 17.171, 4.442, 1.686, 1...  435918.7250   
1  [5.683, 1.002, 3.286, 2.772, 1.682, 1.012, 10....   17948.1780   
2  [21.782, 1.007, 10.21, 3.209, 1.013, 2.786, 2....   19853.3205   
3  [3.002, 47.632, 202.038, 1.379, 2.01, 27.867, ...   15225.3150   
4  [1.608, 1.286, 18.849, 5.328, 52.666, 7.4, 1.6...   32881.7910   

    end_ppl_cg  bt_ins_avg_ppl  ed_ins_avg_ppl  
0  435918.7250      87201.6645       1574.8318  
1   17948.1780       3597.5081       1238.7556  
2   19853.3205       3980.2342       1714.5883  
3   15225.3150       3081.0149      65142.2633  
4   32881.7910 

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7ff9d0f2bca0>>
Traceback (most recent call last):
  File "/opt/conda/envs/pan/lib/python3.10/site-packages/ipykernel/ipkernel.py", line 770, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(
KeyboardInterrupt: 


In [11]:
print(df.shape)

print(df[["bt_ins_avg_ppl", "ed_ins_avg_ppl"]])


(200, 6)
     bt_ins_avg_ppl  ed_ins_avg_ppl
0      8.720166e+04       1574.8318
1      3.597508e+03       1238.7556
2      3.980234e+03       1714.5883
3      3.081015e+03      65142.2633
4      6.591226e+03        282.3941
..              ...             ...
195    3.552773e+03      34413.7137
196    1.922698e+03       9597.3545
197    7.865348e+03      20742.2896
198    9.932671e+03       5408.1822
199    3.189650e+07     604347.1192

[200 rows x 2 columns]
